In [1]:
import pandas as pd
import datetime as dt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA

In [2]:
# Fetching data
start = dt.datetime(2021, 7, 1)
end = dt.datetime(2022, 1, 27)
symbol = 'AXISBANK.NS'
stk_data = yf.download(symbol, start=start, end=end)

[*********************100%%**********************]  1 of 1 completed


In [3]:
# Selecting specific columns
stk_data = stk_data[["Open", "High", "Low", "Close"]]

In [4]:
stk_data

Open        High         Low       Close
Date                                                      
2021-07-01  750.000000  754.400024  744.500000  746.450012
2021-07-02  747.599976  752.849976  744.000000  750.650024
2021-07-05  757.099976  762.000000  753.400024  760.349976
2021-07-06  757.000000  767.000000  755.200012  758.349976
2021-07-07  755.900024  764.000000  752.250000  762.900024
...                ...         ...         ...         ...
2022-01-19  725.200012  731.950012  718.700012  729.599976
2022-01-20  731.049988  737.150024  722.150024  728.900024
2022-01-21  722.000000  725.250000  708.049988  712.500000
2022-01-24  715.000000  716.400024  688.200012  704.599976
2022-01-25  714.000000  755.000000  708.599976  752.200012

[143 rows x 4 columns]

In [5]:
stk_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 143 entries, 2021-07-01 to 2022-01-25
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    143 non-null    float64
 1   High    143 non-null    float64
 2   Low     143 non-null    float64
 3   Close   143 non-null    float64
dtypes: float64(4)
memory usage: 5.6 KB


In [6]:
# Scaling data
column = "Close"
MS = MinMaxScaler()
data1 = MS.fit_transform(stk_data[[column]])

In [7]:
# Splitting the data into train and test sets
training_size = round(len(data1) * 0.80)
X_train = data1[:training_size]
X_test = data1[training_size:]
y_train = data1[:training_size]
y_test = data1[training_size:]

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
# Define trends and lags
#trends = ['n', 't', 'c', 'ct']
orders = [(1,0,1),(1,0,2),(2,0,1),(2,0,2)]

In [10]:
# Function to calculate RMSE and MAPE
def rmsemape(actual, predicted):
    mse = mean_squared_error(actual, predicted, squared=False)
    print(f"RMSE: {mse}")
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    print(f"MAPE: {mape}%")
    return mse, mape

In [14]:
# Training and evaluating models
import numpy as np
for i in orders:
#for td in trends:
    model = ARIMA(X_train,order=i)
    model_fit = model.fit()
    y_pred = model_fit.predict(len(X_train), len(data1) - 1)
    #print(f"Trend={td}")
    rmsemape(y_test, y_pred)
    print("****")

RMSE: 0.18252003308098042
MAPE: 124.06811896945914%
****
RMSE: 0.1910282360518972
MAPE: 129.34870242398247%
****
RMSE: 0.1861623092882079
MAPE: 126.30842814836481%
****
RMSE: 0.1881884432544495
MAPE: 127.87603320513267%
****


In [15]:
# Forecasting
i = (1,0,1)
#td = 'ct'
model = ARIMA(X_train, order=i)
model_fit = model.fit()
y_pred = model_fit.predict(len(X_train), len(data1) - 1)

In [16]:
y_pred

array([0.32699811, 0.33802518, 0.34837104, 0.35807777, 0.36718485,
       0.37572933, 0.38374596, 0.39126735, 0.3983241 , 0.40494491,
       0.41115671, 0.41698476, 0.42245278, 0.427583  , 0.4323963 ,
       0.43691225, 0.44114922, 0.44512444, 0.44885409, 0.45235334,
       0.45563641, 0.45871667, 0.46160664, 0.46431808, 0.46686201,
       0.46924879, 0.47148813, 0.47358912, 0.47556033])

In [17]:
# Inverse transform the scaled data
aTestNormTable = pd.DataFrame(y_test, columns=[column])
actual_stock_price_test_ori = MS.inverse_transform(aTestNormTable)
actual_stock_price_test_oriA = pd.DataFrame(actual_stock_price_test_ori, columns=[column])

In [18]:
actual_stock_price_test_oriA

Close
0   709.650024
1   688.900024
2   674.349976
3   665.849976
4   669.349976
5   678.799988
6   667.500000
7   672.150024
8   675.450012
9   670.500000
10  668.849976
11  678.549988
12  696.349976
13  709.150024
14  726.900024
15  730.299988
16  730.599976
17  742.799988
18  743.250000
19  746.849976
20  740.700012
21  721.700012
22  712.650024
23  725.200012
24  729.599976
25  728.900024
26  712.500000
27  704.599976
28  752.200012

In [19]:
pTestNormTable = pd.DataFrame(y_pred, columns=[column])
predicted_stock_price_test_ori = MS.inverse_transform(pTestNormTable)
predicted_stock_price_test_oriP = pd.DataFrame(predicted_stock_price_test_ori, columns=[column])

In [20]:
predicted_stock_price_test_oriP

Close
0   714.537608
1   716.676860
2   718.683957
3   720.567062
4   722.333836
5   723.991465
6   725.546692
7   727.005842
8   728.374851
9   729.659288
10  730.864377
11  731.995019
12  733.055815
13  734.051078
14  734.984858
15  735.860952
16  736.682924
17  737.454118
18  738.177670
19  738.856523
20  739.493440
21  740.091009
22  740.651664
23  741.177683
24  741.671206
25  742.134241
26  742.568672
27  742.976265
28  743.358679

In [21]:
# Plotting actual vs predicted prices
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(actual_stock_price_test_oriA.values, color='blue', label="Actual")
plt.plot(predicted_stock_price_test_oriP.values, color='red', label="Predicted")
plt.legend()
plt.title("Actual vs Predicted Stock Prices")
plt.xlabel("Days")
plt.ylabel("Prices")
plt.show()

In [22]:
# Forecasting future prices
forecast = model_fit.predict(len(data1), len(data1) + 3)
fTestNormTable = pd.DataFrame(forecast, columns=["Closefore"])
forecast_stock_price_test_ori = MS.inverse_transform(fTestNormTable)
forecast_stock_price_test_oriF = pd.DataFrame(forecast_stock_price_test_ori, columns=["Closefore"])

In [23]:
forecast

array([0.47740975, 0.47914493, 0.48077292, 0.48230033])

In [24]:
fTestNormTable

Closefore
0   0.477410
1   0.479145
2   0.480773
3   0.482300

In [25]:
forecast_stock_price_test_ori

array([[743.71746796],
       [744.05409253],
       [744.36992164],
       [744.66623996]])

In [26]:
forecast_stock_price_test_oriF

Closefore
0  743.717468
1  744.054093
2  744.369922
3  744.666240

In [27]:
# Save forecasted prices to CSV
forecast_stock_price_test_oriF.to_csv("CloseARMA.csv", index=False)

In [28]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))
plt.plot(actual_stock_price_test_oriA[116:145], color = 'blue', label="Actual")
plt.plot(forecast_stock_price_test_oriF, color = 'red', label ="Predicted")